In [1]:
pip install opencv-python mediapipe


     ------------------------------------- 904.2/904.2 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.23.2
    Uninstalling protobuf-4.23.2:
      Successfully uninstalled protobuf-4.23.2
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.11.0 requires keras<2.12,>=2.11.0, but you have keras 2.12.0 which is incompatible.
tensorflow-intel 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorflow-intel 2.11.0 requires tensorflow-estimator<2.12,>=2.11.0, but you have tensorflow-estimator 2.12.0 which is incompatible.
tensorboard 2.11.2 requires tensorboard-data-server<0.7.0,>=0.6.0, but you have tensorboard-data-server 0.7.0 which is incompatible.


In [2]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Hand model
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Initialize canvas
canvas = np.zeros((480, 640, 3), dtype=np.uint8)
canvas_prev = canvas.copy()

# Initialize video capture
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# Parameters for drawing
drawing_color = (0, 255, 0)
drawing_thickness = 3

with mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Flip the frame horizontally for a mirrored view
        frame = cv2.flip(frame, 1)

        # Convert the BGR image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the image with MediaPipe
        results = hands.process(image)

        # Reset canvas if necessary
        if results.multi_hand_landmarks is not None:
            for hand_landmarks in results.multi_hand_landmarks:
                index_finger = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                x = int(index_finger.x * frame.shape[1])
                y = int(index_finger.y * frame.shape[0])

                # Draw on canvas
                cv2.circle(canvas, (x, y), drawing_thickness, drawing_color, -1)

        # Merge the canvas with the frame
        alpha = 0.5
        merged = cv2.addWeighted(canvas, alpha, frame, 1 - alpha, 0)

        # Display the merged frame
        cv2.imshow('Air Canvas', merged)

        # Exit on 'q' press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Hand model
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Initialize canvas
canvas = np.zeros((480, 640, 3), dtype=np.uint8)
canvas_prev = canvas.copy()

# Initialize video capture
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# Parameters for drawing
drawing_color = (0, 255, 0)
drawing_thickness = 3

# Create a window to display camera feed and blank canvas side by side
cv2.namedWindow('Air Canvas', cv2.WINDOW_NORMAL)
cv2.setWindowProperty('Air Canvas', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

# Color selection key mapping
color_mapping = {
    ord('1'): (0, 255, 0),  # Green
    ord('2'): (0, 0, 255),  # Blue
    ord('3'): (255, 0, 0),  # Red
}

with mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Flip the frame horizontally for a mirrored view
        frame = cv2.flip(frame, 1)

        # Convert the BGR image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the image with MediaPipe
        results = hands.process(image)

        # Reset canvas if necessary
        if results.multi_hand_landmarks is not None:
            for hand_landmarks in results.multi_hand_landmarks:
                index_finger = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                x = int(index_finger.x * frame.shape[1])
                y = int(index_finger.y * frame.shape[0])

                # Draw on canvas
                cv2.circle(canvas, (x, y), drawing_thickness, drawing_color, -1)

        # Concatenate the frames horizontally
        output_frame = np.hstack((frame, canvas))

        # Display the combined frame
        cv2.imshow('Air Canvas', output_frame)

        # Update drawing color on key press
        key = cv2.waitKey(1) & 0xFF
        if key in color_mapping:
            drawing_color = color_mapping[key]

        # Reset canvas on 'c' key press
        if key == ord('c'):
            canvas = np.zeros((480, 640, 3), dtype=np.uint8)
            canvas_prev = canvas.copy()

        # Exit on 'q' press
        if key == ord('q'):
            break

# Release resources
cap.release()
cv2.destroyAllWindows()
